In [0]:
# Need to automate the process.
# Instead of reading and writing the files, we can use the following code to read the files and write the files.

import os
from pyspark.sql.functions import current_timestamp, input_file_name

input_path = "/Volumes/real_time_projects/ecommerce_historical/lakehouse_vol/Raw_DS/"
output_base_path = "/Volumes/real_time_projects/ecommerce_historical/lakehouse_vol/bronze/"

# ==============================
# List Input Files
# ==============================
files = dbutils.fs.ls(input_path)

for file in files:
    try:
        file_path = file.path
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        print(f"🚀 Processing file: {file_name}")

        # ==============================
        # Read CSV (Schema inference once per file)
        # ==============================
        df = (
            spark.read
            .format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load(file_path)
        )

        # ==============================
        # Add Audit Columns (Industry Standard)
        # ==============================
        df_enriched = (
            df
            .withColumn("ingestion_timestamp", current_timestamp())
            .withColumn("source_file", input_file_name())
        )

        output_path = f"{output_base_path}/{file_name}"

        print(f"✅ Successfully processed: {file_name}")

    except Exception as e:
        print(f"❌ Failed processing {file_name}: {str(e)}")